# 3.13 高性能Pandas：eval()与query()

## 3.13.1 query()与eval()的设计动机：复合代数式

对下面两个数组进行求和

In [1]:
import numpy as np

In [2]:
rng = np.random.RandomState(42)

In [7]:
x = rng.rand(1000000)

In [8]:
y = rng.rand(1000000)

In [9]:
x

array([0.37454012, 0.95071431, 0.73199394, ..., 0.41807198, 0.42867126,
       0.92944855])

In [10]:
y

array([0.59515562, 0.36471714, 0.00537562, ..., 0.68311082, 0.33865907,
       0.69161641])

In [11]:
%timeit x + y

3.33 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

247 ms ± 4.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
mask = (x > 0.5) & (y < 0.5)

In [14]:
mask

array([False,  True,  True, ..., False, False, False])

In [17]:
import numexpr

In [18]:
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')

In [19]:
mask_numexpr

array([False,  True,  True, ..., False, False, False])

In [20]:
np.allclose(mask, mask_numexpr)

True

## 3.13.2 用pandas.eval()实现高效能运算

In [21]:
import pandas as pd

In [22]:
nrows, ncols = 100000, 100

In [23]:
rng = np.random.RandomState(45)

In [24]:
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [26]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.989012,0.549545,0.281447,0.077290,0.444469,0.472808,0.048522,0.163324,0.115951,0.627392,...,0.684723,0.105235,0.604238,0.737261,0.237222,0.988986,0.840792,0.433100,0.722755,0.668636
1,0.523749,0.297884,0.570986,0.573988,0.044422,0.453265,0.990007,0.226716,0.473247,0.078854,...,0.055044,0.037928,0.104318,0.589035,0.887253,0.155320,0.435512,0.990927,0.381057,0.876464
2,0.686148,0.635905,0.659813,0.951054,0.890984,0.621702,0.843640,0.471274,0.544026,0.866244,...,0.953765,0.863029,0.397317,0.021653,0.800174,0.620063,0.551669,0.595423,0.730732,0.761468
3,0.329780,0.656230,0.482373,0.891246,0.088806,0.833458,0.044397,0.185922,0.654306,0.618251,...,0.401566,0.805070,0.705785,0.358530,0.128620,0.538505,0.927268,0.937177,0.959899,0.789929
4,0.778140,0.284187,0.435468,0.913342,0.289110,0.209037,0.209183,0.411857,0.105141,0.399046,...,0.574667,0.640085,0.074437,0.631337,0.030931,0.584960,0.195182,0.286383,0.980907,0.797602


普通的Pandas方法计算四个DataFrame的和

In [28]:
%timeit df1 + df2 + df3 + df4

90 ms ± 596 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


通过pd.eval和字符串代数式计算并得出相同的结果

In [29]:
%timeit pd.eval('df1 + df2 + df3 + df4')

40.5 ms ± 756 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


eval()版本的代数式比普通方法快一倍（而且内存消耗更少），结果也是一样。

In [30]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

In [31]:
pd.__version__

'0.24.2'

pd.eval()支持的运算

In [32]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3))) for i in range(5))

**（1）算术运算符**

In [33]:
result1 = -df1 * df2 / (df3 + df4) - df5

In [35]:
result1.head()

,0,1,2
0,-1295.789889,-662.524590,-798.124875
1,-712.476662,-877.962963,-851.213656
2,-314.723320,-913.416244,-357.066379
3,-636.995633,-761.851064,-667.380130
4,-425.426745,-500.008143,-637.100000


In [36]:
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')

In [37]:
np.allclose(result1, result2)

True

**（2）比较运算符**

In [47]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)

In [59]:
result2 = pd.eval('df1 < df2 <= df3 != df4')

In [60]:
result1.head()

,0,1,2
0,False,False,False
1,False,False,False
2,False,False,True
3,False,True,False
4,True,False,False


In [61]:
result2.head()

,0,1,2
0,False,False,False
1,False,False,False
2,False,False,True
3,False,True,False
4,True,False,False


In [62]:
np.allclose(result1, result2)

True

**（3）位运算符**

In [63]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)

In [64]:
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')

In [65]:
np.allclose(result1, result2)

True

此外还可以使用and和or

In [66]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')

In [67]:
np.allclose(result1, result3)

True

**（4）对象属性与索引**

In [68]:
result1 = df2.T[0] + df3.iloc[1]

In [72]:
df2.T[0]

0    967
1    964
2    643
Name: 0, dtype: int32

In [73]:
df3.iloc[1]

0    605
1     69
2    393
Name: 1, dtype: int32

In [69]:
result1

0    1572
1    1033
2    1036
dtype: int32

In [74]:
result2 = pd.eval('df2.T[0] + df3.iloc[1]')

In [75]:
np.allclose(result1, result2)

True

## 3.13.3 用DataFrame.eval()实现列间运算